In [1]:
import pandas as pd
from tqdm import tqdm
from mltb2.text import has_xml_tag
from uuid import uuid4

tqdm.pandas()  # activate tqdm pandas progress bar support

In [2]:
df = pd.read_pickle("./data/wikipedia-22-12-de-no-emb-clean-selected-merged.pkl.gz")
df.rename(columns={"text": "context", "token_count": "context_token_count"}, inplace=True)
df

,wiki_id,title,context,context_token_count
0,784611,Friedrich Stromeyer,Friedrich Stromeyer (* 2. August 1776 in Götti...,193
1,5871091,Siddiqiy Ajziy,"Saidahmad Siddiqiy (auch ""Saidahmadxoʻja""; kyr...",253
2,5365282,"7,5-cm-Leichtgeschütz 40","Das 7,5-cm-Leichtgeschütz 40 (kurz ""LG 40"") wa...",266
3,12114918,Tiberius Claudius Claudianus,<onlyinclude>* Tiberius Claudius Claudianus (S...,37
4,7596131,Gmina Stargard,"Die Gmina Stargard (von 1950 bis 2015 ""Stargar...",90
...,...,...,...,...
136714,11341584,Mabel Addis,Addis wurde als Mabel Holmes als Tochter von J...,203
136715,2277242,Robert Beyer (Tonmeister),Robert Beyer gehörte zu jenen radikalen Erneue...,244
136716,7957370,Gabriel Cyrille,Nach der Grundschule in Zinder besuchte Gabrie...,266
136717,3420897,Serbischer Psalter,Der Serbische Psalter (auch Codex Monacensis S...,148


In [3]:
df["has_xml_tag"] = df["context"].progress_apply(has_xml_tag)
df["has_xml_tag"].value_counts(dropna=False)

100%|██████████| 135295/135295 [00:00<00:00, 749051.07it/s]


has_xml_tag
False    134705
True        590
Name: count, dtype: int64

In [4]:
df.drop(df[df["has_xml_tag"] == True].index, inplace=True)
df.drop("has_xml_tag", axis=1, inplace=True)

In [5]:
print(len(df))
df.drop_duplicates("context", inplace=True, keep="first")
print(len(df))

134705
134705


## Insight

- no duplicates available

In [6]:
df["context_uuid"] = df["context"].progress_apply(lambda _: str(uuid4()))
df

100%|██████████| 134705/134705 [00:00<00:00, 418415.31it/s]


,wiki_id,title,context,context_token_count,context_uuid
0,784611,Friedrich Stromeyer,Friedrich Stromeyer (* 2. August 1776 in Götti...,193,031e05f9-030c-496d-81a4-89566f0f1e04
1,5871091,Siddiqiy Ajziy,"Saidahmad Siddiqiy (auch ""Saidahmadxoʻja""; kyr...",253,fa2d553a-114c-49c1-a964-6e6573f0c89a
2,5365282,"7,5-cm-Leichtgeschütz 40","Das 7,5-cm-Leichtgeschütz 40 (kurz ""LG 40"") wa...",266,d46c893e-e6eb-438f-ac6a-72dbd3fe6863
4,7596131,Gmina Stargard,"Die Gmina Stargard (von 1950 bis 2015 ""Stargar...",90,fe251f5f-2552-4115-b01c-52165fb1a17e
5,6396408,Rosetta Tofano,Rosetta Tofano (* 5. März 1902 in Mailand als ...,136,365a414d-0479-4768-b8f5-aa22a142ba3e
...,...,...,...,...,...
136714,11341584,Mabel Addis,Addis wurde als Mabel Holmes als Tochter von J...,203,11081295-fece-4bf2-aacd-df0b07afce74
136715,2277242,Robert Beyer (Tonmeister),Robert Beyer gehörte zu jenen radikalen Erneue...,244,6879bd80-166c-4696-bcbd-fe57ce670b42
136716,7957370,Gabriel Cyrille,Nach der Grundschule in Zinder besuchte Gabrie...,266,69982034-63ff-4f4c-af03-abcd4a388789
136717,3420897,Serbischer Psalter,Der Serbische Psalter (auch Codex Monacensis S...,148,e36778ec-0d1f-4991-9bcd-f80aa131d427


In [7]:
df.to_pickle("./data/wikipedia-22-12-de-context.pkl.gz", compression="gzip")